In [1]:
import jsonpickle
import pandas as pd
import re
import string

In [2]:
import json, os, sys
import snscrape.modules.twitter as sntwitter
import datetime, time, requests
from datetime import datetime 
from datetime import date
from datetime import timedelta

In [3]:
import requests
import time, random
from stem import Signal
from stem.control import Controller
from fake_useragent import UserAgent
import random, time

In [4]:
language = 'lang:zh'  #set language to Chinese
location = '36.186107, 105.163097, 2010km'

In [5]:
proxies = {
    'http': 'socks5://127.0.0.1:9050',
    'https': 'socks5://127.0.0.1:9050'
}

url = 'https://ident.me'

In [6]:
headers = {'User-Agent': UserAgent().random }
requests.get(url, proxies=proxies, headers=headers).text

'87.237.165.31'

In [7]:
def renew_ip():
    with Controller.from_port(port=9051) as c:
        c.authenticate(password='kalilinux')
        c.signal(Signal.NEWNYM)

In [8]:
current_datetime = datetime.now()
print(current_datetime)

2022-09-11 09:29:27.856093


In [9]:
date = date.today()
print(date)

2022-09-11


In [10]:
previous_hour = current_datetime - timedelta(hours=1)
print(previous_hour)

2022-09-11 08:29:27.856093


In [11]:
Tweets = "ChineseTweets" + str(date) + ".json"

In [12]:
tweets = []

## Had trouble webscraping the tweets into JSON, but I'll make sure to solve that 

In [13]:
for r in range (10):
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(language, 'since:previous_hour until:current_datetime').get_items()):
        if i>=50:
            break
        tweets.append({"tweet":tweet, "date":tweet.date, "id":tweet.id, "content":tweet.content, "user":tweet.user.username})
        
        #tweets.append({"tweet":tweet, "date":tweet.date, "id":tweet.id, "content":tweet.content, "user":tweet.user.username})
        #with open(Tweets, 'w', encoding='utf-8') as f:json.dumps({"tweet":tweet, "date":tweet.date, "id":tweet.id, "content":tweet.content, "user":tweet.user.username})

In [14]:
print(len(tweets))

500


In [15]:
type(tweets)

list

In [16]:
tweets

[{'tweet': Tweet(url='https://twitter.com/fangyuan2004/status/1568954906111000577', date=datetime.datetime(2022, 9, 11, 13, 29, 34, tzinfo=datetime.timezone.utc), content='这是在干嘛😏 https://t.co/pGTLktRnRo', renderedContent='这是在干嘛😏 twitter.com/ohmpawatt/stat…', id=1568954906111000577, user=User(username='fangyuan2004', id=1488517007582453761, displayname='不嗑会很烦的(:(', description='🇨🇳🇹🇭', rawDescription='🇨🇳🇹🇭', descriptionUrls=None, verified=False, created=datetime.datetime(2022, 2, 1, 14, 18, 17, tzinfo=datetime.timezone.utc), followersCount=0, friendsCount=19, statusesCount=162, favouritesCount=464, listedCount=0, mediaCount=0, location='', protected=False, linkUrl=None, linkTcourl=None, profileImageUrl='https://pbs.twimg.com/profile_images/1567199332457402371/GZl0_1A6_normal.jpg', profileBannerUrl='https://pbs.twimg.com/profile_banners/1488517007582453761/1662168737', label=None), replyCount=0, retweetCount=0, likeCount=0, quoteCount=0, conversationId=1568954906111000577, lang='zh', sour

In [17]:
Tweets = pd.DataFrame.from_records(tweets)

In [18]:
Tweets['content'][0]

'这是在干嘛😏 https://t.co/pGTLktRnRo'

In [19]:
Tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype              
---  ------   --------------  -----              
 0   tweet    500 non-null    object             
 1   date     500 non-null    datetime64[ns, UTC]
 2   id       500 non-null    int64              
 3   content  500 non-null    object             
 4   user     500 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(3)
memory usage: 19.7+ KB


*****************************************************************************************************

### Preprocessing

#### Removing urls

In [20]:
def remove_urls(text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    return(text)

In [21]:
%time
Tweets['content'] = Tweets['content'].apply(lambda x:remove_urls(x))

CPU times: total: 0 ns
Wall time: 0 ns


#### Removing @

In [22]:
def strip_all_entities(text):
    entity_prefixes = ['@']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [23]:
%time
Tweets['content'] = Tweets['content'].apply(lambda x:strip_all_entities(x))

CPU times: total: 0 ns
Wall time: 0 ns


#### Remove emojis

In [24]:
def de_emojify(text):
    regex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
                    
                           "]+", flags = re.UNICODE)
    return regex_pattern.sub(r'', text)

In [25]:
%time
Tweets['content'] = Tweets['content'].apply(lambda x:de_emojify(x))

CPU times: total: 0 ns
Wall time: 0 ns


In [26]:
Tweets

,tweet,date,id,content,user
0,https://twitter.com/fangyuan2004/status/156895...,2022-09-11 13:29:34+00:00,1568954906111000577,这是在干嘛,fangyuan2004
1,https://twitter.com/Bacon_7777/status/15689549...,2022-09-11 13:29:33+00:00,1568954905809027072,赤子孤独了，会创造一个世界，创造许多心灵的朋友 BadBuddyRerunEP2,Bacon_7777
2,https://twitter.com/nmsl_5450/status/156895490...,2022-09-11 13:29:33+00:00,1568954904982470656,我想死。我也许活不到下周去玩了,nmsl_5450
3,https://twitter.com/LauDouglas2/status/1568954...,2022-09-11 13:29:33+00:00,1568954903187554304,谁懂啊，朋友的现任女友是我前任炮友关键是我朋友还老在微信上发他和她女朋友的照片,LauDouglas2
4,https://twitter.com/shunjihioki/status/1568954...,2022-09-11 13:29:33+00:00,1568954903145623554,曼殊沙華 ⇒ アメブロ via official,shunjihioki
...,...,...,...,...,...
495,https://twitter.com/negi_bana_sword/status/156...,2022-09-11 13:29:51+00:00,1568954980106633218,米克絲「啊奧利佛，你不用這麼難過的，我只是在教導我的同伴，要用正確的心態跟別人相處而已」,negi_bana_sword
496,https://twitter.com/sola10505/status/156895498...,2022-09-11 13:29:51+00:00,1568954980077535232,好耶暈爛YA呼 出航出航,sola10505
497,https://twitter.com/klomkiew/status/1568954979...,2022-09-11 13:29:51+00:00,1568954979989491715,长大了不可以随便玩了，以前和现在不一样了【好！记得自己说的话！】 想改变也可以不要后悔啊,klomkiew
498,https://twitter.com/dyingblanket/status/156895...,2022-09-11 13:29:51+00:00,1568954979809136643,今天出國明天回來，18小時極限燒錢盤子挑戰【暴發互 5】｜HOOK ft 台啤18天生啤酒 ...,dyingblanket


#### Removing chinese punctuation

In [27]:
def remove_chinese_punctuation(text):
    text = re.sub("[！？。｡＂＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃《》「」『』【】〔〕\
                〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.!\"$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+", "",text) 
    return text

In [28]:
%time
Tweets['content'] = Tweets['content'].apply(lambda x:remove_chinese_punctuation(x))

CPU times: total: 0 ns
Wall time: 0 ns


In [29]:
Tweets['content'][496]

'好耶暈爛YA呼出航出航'

#### Removing english characters

In [30]:
def escape_en_char(text):
    en_pattern = "[a-zA-Z0-9]+"
    text = re.sub(en_pattern, ' ', str(text))
    return text

In [31]:
%time
Tweets['content'] = Tweets['content'].apply(lambda x:escape_en_char(x))

CPU times: total: 0 ns
Wall time: 0 ns


In [32]:
Tweets

,tweet,date,id,content,user
0,https://twitter.com/fangyuan2004/status/156895...,2022-09-11 13:29:34+00:00,1568954906111000577,这是在干嘛,fangyuan2004
1,https://twitter.com/Bacon_7777/status/15689549...,2022-09-11 13:29:33+00:00,1568954905809027072,赤子孤独了会创造一个世界创造许多心灵的朋友,Bacon_7777
2,https://twitter.com/nmsl_5450/status/156895490...,2022-09-11 13:29:33+00:00,1568954904982470656,我想死我也许活不到下周去玩了,nmsl_5450
3,https://twitter.com/LauDouglas2/status/1568954...,2022-09-11 13:29:33+00:00,1568954903187554304,谁懂啊朋友的现任女友是我前任炮友关键是我朋友还老在微信上发他和她女朋友的照片,LauDouglas2
4,https://twitter.com/shunjihioki/status/1568954...,2022-09-11 13:29:33+00:00,1568954903145623554,曼殊沙華⇒アメブロ,shunjihioki
...,...,...,...,...,...
495,https://twitter.com/negi_bana_sword/status/156...,2022-09-11 13:29:51+00:00,1568954980106633218,米克絲啊奧利佛你不用這麼難過的我只是在教導我的同伴要用正確的心態跟別人相處而已,negi_bana_sword
496,https://twitter.com/sola10505/status/156895498...,2022-09-11 13:29:51+00:00,1568954980077535232,好耶暈爛 呼出航出航,sola10505
497,https://twitter.com/klomkiew/status/1568954979...,2022-09-11 13:29:51+00:00,1568954979989491715,长大了不可以随便玩了以前和现在不一样了好记得自己说的话想改变也可以不要后悔啊,klomkiew
498,https://twitter.com/dyingblanket/status/156895...,2022-09-11 13:29:51+00:00,1568954979809136643,今天出國明天回來 小時極限燒錢盤子挑戰暴發互 台啤 天生啤酒來自好想去泰國住貴飯店,dyingblanket


In [33]:
Tweets1 = Tweets.copy(deep=True)

*****************************************************************************************************

In [34]:
import spacy
import pytextrank
from collections import Counter
import jieba
import jieba.posseg as pseg
from spacy.lang.zh.stop_words import STOP_WORDS
nlp = spacy.load('zh_core_web_sm')
nlp.add_pipe("textrank", last=True)

C:\Users\Usuario\anaconda3\envs\spyder1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The following cell creates a new column with tokenized content, its commented because later a loop iterates over the column and applies text rank

In [35]:
#Tweets['tokenized_content'] = Tweets['content'].apply(lambda x: nlp(x))

In [36]:
Tweets

,tweet,date,id,content,user,tokenized_content
0,https://twitter.com/fangyuan2004/status/156895...,2022-09-11 13:29:34+00:00,1568954906111000577,这是在干嘛,fangyuan2004,"(这是, 在, 干嘛)"
1,https://twitter.com/Bacon_7777/status/15689549...,2022-09-11 13:29:33+00:00,1568954905809027072,赤子孤独了会创造一个世界创造许多心灵的朋友,Bacon_7777,"(赤子, 孤独, 了, 会, 创造, 一个, 世界, 创造, 许多, 心灵, 的, 朋友)"
2,https://twitter.com/nmsl_5450/status/156895490...,2022-09-11 13:29:33+00:00,1568954904982470656,我想死我也许活不到下周去玩了,nmsl_5450,"(我, 想, 死, 我, 也许, 活, 不, 到, 下周, 去, 玩, 了)"
3,https://twitter.com/LauDouglas2/status/1568954...,2022-09-11 13:29:33+00:00,1568954903187554304,谁懂啊朋友的现任女友是我前任炮友关键是我朋友还老在微信上发他和她女朋友的照片,LauDouglas2,"(谁, 懂, 啊, 朋友, 的, 现任, 女友, 是, 我, 前任, 炮友, 关键, 是, ..."
4,https://twitter.com/shunjihioki/status/1568954...,2022-09-11 13:29:33+00:00,1568954903145623554,曼殊沙華⇒アメブロ,shunjihioki,"(曼殊, 沙華, ⇒ア, メブ, ロ)"
...,...,...,...,...,...,...
495,https://twitter.com/negi_bana_sword/status/156...,2022-09-11 13:29:51+00:00,1568954980106633218,米克絲啊奧利佛你不用這麼難過的我只是在教導我的同伴要用正確的心態跟別人相處而已,negi_bana_sword,"(米克絲, 啊, 奧利佛, 你, 不用, 這麼, 難過, 的, 我, 只是, 在, 教導, ..."
496,https://twitter.com/sola10505/status/156895498...,2022-09-11 13:29:51+00:00,1568954980077535232,好耶暈爛 呼出航出航,sola10505,"(好, 耶暈, 爛, 呼出, 航出, 航)"
497,https://twitter.com/klomkiew/status/1568954979...,2022-09-11 13:29:51+00:00,1568954979989491715,长大了不可以随便玩了以前和现在不一样了好记得自己说的话想改变也可以不要后悔啊,klomkiew,"(长大, 了, 不, 可以, 随便, 玩, 了, 以前, 和, 现在, 不, 一样, 了, ..."
498,https://twitter.com/dyingblanket/status/156895...,2022-09-11 13:29:51+00:00,1568954979809136643,今天出國明天回來 小時極限燒錢盤子挑戰暴發互 台啤 天生啤酒來自好想去泰國住貴飯店,dyingblanket,"(今天, 出國, 明天, 回來, 小時, 極限, 燒錢, 盤子, 挑戰, 暴發, 互, 台啤..."


*******************************************************************************************************************************

In [37]:
pytext_key = []

for text in Tweets['content']:
    text = nlp(text)
    t = text._.phrases
    pytext_key.append(t)
    
Tweets['Pytextrank'] = pytext_key  

In [38]:
Tweets

,tweet,date,id,content,user,tokenized_content,Pytextrank
0,https://twitter.com/fangyuan2004/status/156895...,2022-09-11 13:29:34+00:00,1568954906111000577,这是在干嘛,fangyuan2004,"(这是, 在, 干嘛)",[]
1,https://twitter.com/Bacon_7777/status/15689549...,2022-09-11 13:29:33+00:00,1568954905809027072,赤子孤独了会创造一个世界创造许多心灵的朋友,Bacon_7777,"(赤子, 孤独, 了, 会, 创造, 一个, 世界, 创造, 许多, 心灵, 的, 朋友)",[]
2,https://twitter.com/nmsl_5450/status/156895490...,2022-09-11 13:29:33+00:00,1568954904982470656,我想死我也许活不到下周去玩了,nmsl_5450,"(我, 想, 死, 我, 也许, 活, 不, 到, 下周, 去, 玩, 了)","[Phrase(text='下周', chunks=[下周], count=1, rank=..."
3,https://twitter.com/LauDouglas2/status/1568954...,2022-09-11 13:29:33+00:00,1568954903187554304,谁懂啊朋友的现任女友是我前任炮友关键是我朋友还老在微信上发他和她女朋友的照片,LauDouglas2,"(谁, 懂, 啊, 朋友, 的, 现任, 女友, 是, 我, 前任, 炮友, 关键, 是, ...",[]
4,https://twitter.com/shunjihioki/status/1568954...,2022-09-11 13:29:33+00:00,1568954903145623554,曼殊沙華⇒アメブロ,shunjihioki,"(曼殊, 沙華, ⇒ア, メブ, ロ)",[]
...,...,...,...,...,...,...,...
495,https://twitter.com/negi_bana_sword/status/156...,2022-09-11 13:29:51+00:00,1568954980106633218,米克絲啊奧利佛你不用這麼難過的我只是在教導我的同伴要用正確的心態跟別人相處而已,negi_bana_sword,"(米克絲, 啊, 奧利佛, 你, 不用, 這麼, 難過, 的, 我, 只是, 在, 教導, ...","[Phrase(text='米克絲', chunks=[米克絲], count=1, ran..."
496,https://twitter.com/sola10505/status/156895498...,2022-09-11 13:29:51+00:00,1568954980077535232,好耶暈爛 呼出航出航,sola10505,"(好, 耶暈, 爛, 呼出, 航出, 航)",[]
497,https://twitter.com/klomkiew/status/1568954979...,2022-09-11 13:29:51+00:00,1568954979989491715,长大了不可以随便玩了以前和现在不一样了好记得自己说的话想改变也可以不要后悔啊,klomkiew,"(长大, 了, 不, 可以, 随便, 玩, 了, 以前, 和, 现在, 不, 一样, 了, ...",[]
498,https://twitter.com/dyingblanket/status/156895...,2022-09-11 13:29:51+00:00,1568954979809136643,今天出國明天回來 小時極限燒錢盤子挑戰暴發互 台啤 天生啤酒來自好想去泰國住貴飯店,dyingblanket,"(今天, 出國, 明天, 回來, 小時, 極限, 燒錢, 盤子, 挑戰, 暴發, 互, 台啤...","[Phrase(text='台啤 天生啤酒', chunks=[台啤 天生啤酒], coun..."


In [39]:
Tweets['Pytextrank'][1]

[]

In [40]:
empty_lists = Tweets[Tweets['Pytextrank'].apply( len ) == 0].index

In [41]:
print(len(empty_lists))

282


#### Testing if individual rows arent detected by pytextrank

In [42]:
doc = nlp(Tweets['content'][1])

In [43]:
for phrase in doc._.phrases:
    print("%s %s %s"%(phrase.rank, phrase.count, phrase.text))

*******************************************************************************************************************************

In [44]:
import collections

In [45]:
Tweets

,tweet,date,id,content,user,tokenized_content,Pytextrank
0,https://twitter.com/fangyuan2004/status/156895...,2022-09-11 13:29:34+00:00,1568954906111000577,这是在干嘛,fangyuan2004,"(这是, 在, 干嘛)",[]
1,https://twitter.com/Bacon_7777/status/15689549...,2022-09-11 13:29:33+00:00,1568954905809027072,赤子孤独了会创造一个世界创造许多心灵的朋友,Bacon_7777,"(赤子, 孤独, 了, 会, 创造, 一个, 世界, 创造, 许多, 心灵, 的, 朋友)",[]
2,https://twitter.com/nmsl_5450/status/156895490...,2022-09-11 13:29:33+00:00,1568954904982470656,我想死我也许活不到下周去玩了,nmsl_5450,"(我, 想, 死, 我, 也许, 活, 不, 到, 下周, 去, 玩, 了)","[Phrase(text='下周', chunks=[下周], count=1, rank=..."
3,https://twitter.com/LauDouglas2/status/1568954...,2022-09-11 13:29:33+00:00,1568954903187554304,谁懂啊朋友的现任女友是我前任炮友关键是我朋友还老在微信上发他和她女朋友的照片,LauDouglas2,"(谁, 懂, 啊, 朋友, 的, 现任, 女友, 是, 我, 前任, 炮友, 关键, 是, ...",[]
4,https://twitter.com/shunjihioki/status/1568954...,2022-09-11 13:29:33+00:00,1568954903145623554,曼殊沙華⇒アメブロ,shunjihioki,"(曼殊, 沙華, ⇒ア, メブ, ロ)",[]
...,...,...,...,...,...,...,...
495,https://twitter.com/negi_bana_sword/status/156...,2022-09-11 13:29:51+00:00,1568954980106633218,米克絲啊奧利佛你不用這麼難過的我只是在教導我的同伴要用正確的心態跟別人相處而已,negi_bana_sword,"(米克絲, 啊, 奧利佛, 你, 不用, 這麼, 難過, 的, 我, 只是, 在, 教導, ...","[Phrase(text='米克絲', chunks=[米克絲], count=1, ran..."
496,https://twitter.com/sola10505/status/156895498...,2022-09-11 13:29:51+00:00,1568954980077535232,好耶暈爛 呼出航出航,sola10505,"(好, 耶暈, 爛, 呼出, 航出, 航)",[]
497,https://twitter.com/klomkiew/status/1568954979...,2022-09-11 13:29:51+00:00,1568954979989491715,长大了不可以随便玩了以前和现在不一样了好记得自己说的话想改变也可以不要后悔啊,klomkiew,"(长大, 了, 不, 可以, 随便, 玩, 了, 以前, 和, 现在, 不, 一样, 了, ...",[]
498,https://twitter.com/dyingblanket/status/156895...,2022-09-11 13:29:51+00:00,1568954979809136643,今天出國明天回來 小時極限燒錢盤子挑戰暴發互 台啤 天生啤酒來自好想去泰國住貴飯店,dyingblanket,"(今天, 出國, 明天, 回來, 小時, 極限, 燒錢, 盤子, 挑戰, 暴發, 互, 台啤...","[Phrase(text='台啤 天生啤酒', chunks=[台啤 天生啤酒], coun..."


*******************************************************************************************************************************

On how to count the number of NOUNS in Spacy from a dataframe column: https://stackoverflow.com/questions/71664985/how-to-count-the-number-of-nouns-from-spacy-from-a-dataframe-column

In [46]:
def count_noun(x):
    nouns = [token.text for token in nlp(x) if token.pos_=='NOUN']
    return [nouns, len(nouns)]

Tweets[['list_NOUN','_NOUN']] = pd.DataFrame(Tweets['content'].apply(count_noun).tolist())

In [47]:
Tweets

,tweet,date,id,content,user,tokenized_content,Pytextrank,list_NOUN,_NOUN
0,https://twitter.com/fangyuan2004/status/156895...,2022-09-11 13:29:34+00:00,1568954906111000577,这是在干嘛,fangyuan2004,"(这是, 在, 干嘛)",[],[],0
1,https://twitter.com/Bacon_7777/status/15689549...,2022-09-11 13:29:33+00:00,1568954905809027072,赤子孤独了会创造一个世界创造许多心灵的朋友,Bacon_7777,"(赤子, 孤独, 了, 会, 创造, 一个, 世界, 创造, 许多, 心灵, 的, 朋友)",[],"[世界, 心灵, 朋友]",3
2,https://twitter.com/nmsl_5450/status/156895490...,2022-09-11 13:29:33+00:00,1568954904982470656,我想死我也许活不到下周去玩了,nmsl_5450,"(我, 想, 死, 我, 也许, 活, 不, 到, 下周, 去, 玩, 了)","[Phrase(text='下周', chunks=[下周], count=1, rank=...",[下周],1
3,https://twitter.com/LauDouglas2/status/1568954...,2022-09-11 13:29:33+00:00,1568954903187554304,谁懂啊朋友的现任女友是我前任炮友关键是我朋友还老在微信上发他和她女朋友的照片,LauDouglas2,"(谁, 懂, 啊, 朋友, 的, 现任, 女友, 是, 我, 前任, 炮友, 关键, 是, ...",[],"[朋友, 女友, 前任, 关键, 朋友, 微信, 女朋友, 照片]",8
4,https://twitter.com/shunjihioki/status/1568954...,2022-09-11 13:29:33+00:00,1568954903145623554,曼殊沙華⇒アメブロ,shunjihioki,"(曼殊, 沙華, ⇒ア, メブ, ロ)",[],"[曼殊, ⇒ア, メブ]",3
...,...,...,...,...,...,...,...,...,...
495,https://twitter.com/negi_bana_sword/status/156...,2022-09-11 13:29:51+00:00,1568954980106633218,米克絲啊奧利佛你不用這麼難過的我只是在教導我的同伴要用正確的心態跟別人相處而已,negi_bana_sword,"(米克絲, 啊, 奧利佛, 你, 不用, 這麼, 難過, 的, 我, 只是, 在, 教導, ...","[Phrase(text='米克絲', chunks=[米克絲], count=1, ran...","[這麼, 難過, 同伴, 心態, 別人]",5
496,https://twitter.com/sola10505/status/156895498...,2022-09-11 13:29:51+00:00,1568954980077535232,好耶暈爛 呼出航出航,sola10505,"(好, 耶暈, 爛, 呼出, 航出, 航)",[],[航出],1
497,https://twitter.com/klomkiew/status/1568954979...,2022-09-11 13:29:51+00:00,1568954979989491715,长大了不可以随便玩了以前和现在不一样了好记得自己说的话想改变也可以不要后悔啊,klomkiew,"(长大, 了, 不, 可以, 随便, 玩, 了, 以前, 和, 现在, 不, 一样, 了, ...",[],"[以前, 现在]",2
498,https://twitter.com/dyingblanket/status/156895...,2022-09-11 13:29:51+00:00,1568954979809136643,今天出國明天回來 小時極限燒錢盤子挑戰暴發互 台啤 天生啤酒來自好想去泰國住貴飯店,dyingblanket,"(今天, 出國, 明天, 回來, 小時, 極限, 燒錢, 盤子, 挑戰, 暴發, 互, 台啤...","[Phrase(text='台啤 天生啤酒', chunks=[台啤 天生啤酒], coun...","[今天, 明天, 小時, 天生, 啤酒, 貴飯, 店]",7


*******************************************************************************************************************************

In [48]:
#Function to convert lists inside a dataframe colum into string

#Tweets['list_NOUN_string'] = [','.join(map(str, l)) for l in Tweets['list_NOUN']]

In [49]:
Tweets

,tweet,date,id,content,user,tokenized_content,Pytextrank,list_NOUN,_NOUN
0,https://twitter.com/fangyuan2004/status/156895...,2022-09-11 13:29:34+00:00,1568954906111000577,这是在干嘛,fangyuan2004,"(这是, 在, 干嘛)",[],[],0
1,https://twitter.com/Bacon_7777/status/15689549...,2022-09-11 13:29:33+00:00,1568954905809027072,赤子孤独了会创造一个世界创造许多心灵的朋友,Bacon_7777,"(赤子, 孤独, 了, 会, 创造, 一个, 世界, 创造, 许多, 心灵, 的, 朋友)",[],"[世界, 心灵, 朋友]",3
2,https://twitter.com/nmsl_5450/status/156895490...,2022-09-11 13:29:33+00:00,1568954904982470656,我想死我也许活不到下周去玩了,nmsl_5450,"(我, 想, 死, 我, 也许, 活, 不, 到, 下周, 去, 玩, 了)","[Phrase(text='下周', chunks=[下周], count=1, rank=...",[下周],1
3,https://twitter.com/LauDouglas2/status/1568954...,2022-09-11 13:29:33+00:00,1568954903187554304,谁懂啊朋友的现任女友是我前任炮友关键是我朋友还老在微信上发他和她女朋友的照片,LauDouglas2,"(谁, 懂, 啊, 朋友, 的, 现任, 女友, 是, 我, 前任, 炮友, 关键, 是, ...",[],"[朋友, 女友, 前任, 关键, 朋友, 微信, 女朋友, 照片]",8
4,https://twitter.com/shunjihioki/status/1568954...,2022-09-11 13:29:33+00:00,1568954903145623554,曼殊沙華⇒アメブロ,shunjihioki,"(曼殊, 沙華, ⇒ア, メブ, ロ)",[],"[曼殊, ⇒ア, メブ]",3
...,...,...,...,...,...,...,...,...,...
495,https://twitter.com/negi_bana_sword/status/156...,2022-09-11 13:29:51+00:00,1568954980106633218,米克絲啊奧利佛你不用這麼難過的我只是在教導我的同伴要用正確的心態跟別人相處而已,negi_bana_sword,"(米克絲, 啊, 奧利佛, 你, 不用, 這麼, 難過, 的, 我, 只是, 在, 教導, ...","[Phrase(text='米克絲', chunks=[米克絲], count=1, ran...","[這麼, 難過, 同伴, 心態, 別人]",5
496,https://twitter.com/sola10505/status/156895498...,2022-09-11 13:29:51+00:00,1568954980077535232,好耶暈爛 呼出航出航,sola10505,"(好, 耶暈, 爛, 呼出, 航出, 航)",[],[航出],1
497,https://twitter.com/klomkiew/status/1568954979...,2022-09-11 13:29:51+00:00,1568954979989491715,长大了不可以随便玩了以前和现在不一样了好记得自己说的话想改变也可以不要后悔啊,klomkiew,"(长大, 了, 不, 可以, 随便, 玩, 了, 以前, 和, 现在, 不, 一样, 了, ...",[],"[以前, 现在]",2
498,https://twitter.com/dyingblanket/status/156895...,2022-09-11 13:29:51+00:00,1568954979809136643,今天出國明天回來 小時極限燒錢盤子挑戰暴發互 台啤 天生啤酒來自好想去泰國住貴飯店,dyingblanket,"(今天, 出國, 明天, 回來, 小時, 極限, 燒錢, 盤子, 挑戰, 暴發, 互, 台啤...","[Phrase(text='台啤 天生啤酒', chunks=[台啤 天生啤酒], coun...","[今天, 明天, 小時, 天生, 啤酒, 貴飯, 店]",7


*******************************************************************************************************************************

*******************************************************************************************************************************